In [52]:
save_dir = 'D:/Users/HYUN/Desktop/News_Timeline/data'

In [53]:
import time, os
from selenium import webdriver
import requests
from bs4 import BeautifulSoup

from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.alert import Alert
import time

from pandas import Series as Series
from pandas import DataFrame as DataFrame

## 1. 검찰,법원

In [44]:
# 데이터 프레임 생성
law_df = DataFrame(columns=['date', 'title','body', 'category'])

# 드라이버 지정
chrome_path = 'D:/Users/HYUN/Downloads/chromedriver_win32 (1)/chromedriver.exe'
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument('headless') # headless 모드 설정 (창 안띄움)
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("—disable-extensions")
driver = webdriver.Chrome(chrome_path, options=options)

# 맨 처음 검색할 날짜 지정
date = "2020-06-15"

# 몇일동안 크롤링할지
#date_cnt = 0

# 수집한 기사 개수 카운트
article_cnt = 0

while (article_cnt < 500):
    # 먼저 초기화하고 while문 안에서 page수 계산한 다음 그만큼 돈다
    p = 1
    pages = 1
    # 해당 날짜에 나온 기사의 여러 페이지 반복탐색
    while (p<=pages):
        url = "https://news.joins.com/society/law/list/{}?filter=OnlyJoongang&Date={}".format(p, date)
        driver.get(url)
#         driver.maximize_window()
        response = requests.get(url=url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        pages = len(driver.find_elements_by_class_name("link_page"))
        
        # t_len: 한 페이지 안의 총 기사 개수
        t_len = len(driver.find_elements_by_css_selector("#content > div.list_basic > ul > li"))
        
        # 한 페이지 안의 기사 수 만큼 탐색
        for i in range(1, t_len+1):
            if t_len==1:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li > h2 > a")
            else:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li:nth-child({}) > h2 > a".format(i))
                
            # 제목 수집
            tit = a.text
            
            # 개별 기사 클릭
            a.send_keys(Keys.ENTER)
                
            # 본문 수집
            body = WebDriverWait(driver, 10).until(lambda x : x.find_element_by_css_selector("#article_body")).text
                
            # 데이터 프레임에 삽입
            law_df.loc[article_cnt] = [date, tit, body, 1]
            article_cnt+=1
            
            print(date)
            print(tit)
            print("------------------------------------------------")
                
            # 2500개가 돼도 페이지 안의 기사 수 만큼 for문을 돌기 때문에 2500개일 때 break
            if article_cnt==2500:
                break
                
            driver.back()
            driver.implicitly_wait(3)
            

        p+=1
        if article_cnt==2500:
                break

    if article_cnt==2500:
                break
    # 이전 날짜로 가는 버튼 클릭
    action=ActionChains(driver)
    #h2 = WebDriverWait(driver,10).until(lambda x: x.find_element_by_css_selector("body > div.doc > header > div.mh > div > h2"))
    driver.execute_script("window.scrollTo(0, 0)")
    btn = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_prev")))
    #print(btn)
    btn.send_keys(Keys.ENTER)
    date = WebDriverWait(driver,10).until(lambda x : x.find_element_by_id("display_search_date")).text.replace(".","-")
    
    #date_cnt+=1

driver.close()

law_df.to_csv(save_dir+'law_df')

2020-06-15
'국정농단' 최순실, 추징금 63억 완납···벌금 200억 납부는
------------------------------------------------
2020-06-15
검찰, ‘노조와해 공작 개입’ 삼성전자 임직원들 2심도 실형 구형
------------------------------------------------
2020-06-15
中 보이스피싱 조직 가담해 4억 뜯은 남자…징역 8년
------------------------------------------------
2020-06-15
‘이재용 수사심의위’ 26일 열린다…검찰·변호인 재격돌
------------------------------------------------
2020-06-15
경찰, 동물학대 의혹 ‘갑수목장’ 운영자 소환…계좌 압수수색도
------------------------------------------------
2020-06-15
SNS서 여학생 외모 평가한 남학생에 학교 징계…법원은 "무효"
------------------------------------------------
2020-06-15
유명 마스크업체 가장해 사기친 일당 붙잡아
------------------------------------------------
2020-06-15
쇠사슬 묶는 학대 해놓고…창녕 9살 계부 "딸 많이 사랑해"
------------------------------------------------
2020-06-15
불법촬영 고소한 옛 연인 찾아가 흉기로 찌른 60대
------------------------------------------------
2020-06-15
대검 감찰부장이 SNS에 “한명숙 사건 진상조사 불가피”
------------------------------------------------
2020-06-14
최순실, 옥중 회고록 반응 좋단 말에 눈물 "다행이에요, 다행"
----------------------------

In [45]:
law_df

,date,title,body,category
0,2020-06-15,"'국정농단' 최순실, 추징금 63억 완납···벌금 200억 납부는","\n'국정농단' 사건으로 대법원에서 징역 18년에 벌금 200억원, 추징금 63억...",1
1,2020-06-15,"검찰, ‘노조와해 공작 개입’ 삼성전자 임직원들 2심도 실형 구형",삼성전자서비스 노조와해 혐의로 재판에 넘겨진 이상훈 삼성전자 이사회 의장(왼쪽부터)...,1
2,2020-06-15,中 보이스피싱 조직 가담해 4억 뜯은 남자…징역 8년,보이스피싱 그래픽. 중앙포토\n중국의 전화금융사기(보이스피싱) 조직에 가담해 2년 ...,1
3,2020-06-15,‘이재용 수사심의위’ 26일 열린다…검찰·변호인 재격돌,이재용 삼성전자 부회장. 뉴시스\n이재용(52) 삼성전자 부회장의 기소 타당성을 판...,1
4,2020-06-15,"경찰, 동물학대 의혹 ‘갑수목장’ 운영자 소환…계좌 압수수색도",경찰이 최근 동물학대 의혹을 받은 유튜브 채널 ‘갑수목장’의 운영자 박모(26)씨를...,1
...,...,...,...,...
505,2020-04-24,항소심 “1심 형량 가볍다”…아동 성착취물 구매자에 벌금 200만원,다크웹에서 아동 성착취물을 내려 받아 소지한 20대가 항소심에서 벌금형을 받았다. ...,1
506,2020-04-24,"""회사를 ATM처럼 이용""···800억 횡령 리드 전 부회장 징역8년",서울남부지방법원. 뉴스1\n라임자산운용 이종필 전 부사장과 짜고 회삿돈 수백억원을 ...,1
507,2020-04-24,"사무실 압수수색 당한 황운하 ""경선 상대 후보측이 고발한 듯""",검찰이 24일 더불어민주당 소속 황운하(58·대전 중구) 당선인 사무실을 압수 수색...,1
508,2020-04-24,"‘어지럼증·구토 입원’ MB, 상태 호전돼 오늘 퇴원한다",이명박 전 대통령. 연합뉴스\n \n어지럼증과 구토로 서울대병원에 입원했던 이명박 ...,1


In [50]:
law_df.to_csv('D:/Users/HYUN/Desktop/News_Timeline/data/law_df_8.csv',encoding='utf-8-sig')

In [51]:
import pandas as pd

sum=0
for i in range(1, 9):
    df = pd.read_csv('D:/Users/HYUN/Desktop/News_Timeline/data/law_df_{}.csv'.format(i))
    sum += len(df)
    
print(sum)

2588


## 2. 교육

In [71]:
# 데이터 프레임 생성
education_df = DataFrame(columns=['date', 'title','body', 'category'])

# 드라이버 지정
chrome_path = 'D:/Users/HYUN/Downloads/chromedriver_win32 (1)/chromedriver.exe'
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument('headless') # headless 모드 설정 (창 안띄움)
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("—disable-extensions")
driver = webdriver.Chrome(chrome_path, options=options)

# 맨 처음 검색할 날짜 지정
date = "2020-04-26"

# 몇일동안 크롤링할지
#date_cnt = 0

# 수집한 기사 개수 카운트
article_cnt = 0

while (article_cnt < 100):
    # 먼저 초기화하고 while문 안에서 page수 계산한 다음 그만큼 돈다
    p = 1
    pages = 1
    # 해당 날짜에 나온 기사의 여러 페이지 반복탐색
    while (p<=pages):
        url = "https://news.joins.com/society/education/list/{}?filter=OnlyJoongang&Date={}".format(p, date)
        driver.get(url)
#         driver.maximize_window()
        response = requests.get(url=url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        pages = len(driver.find_elements_by_class_name("link_page"))
        
        # t_len: 한 페이지 안의 총 기사 개수
        t_len = len(driver.find_elements_by_css_selector("#content > div.list_basic > ul > li"))
        
        # 한 페이지 안의 기사 수 만큼 탐색
        for i in range(1, t_len+1):
            if t_len==1:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li > h2 > a")
            else:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li:nth-child({}) > h2 > a".format(i))
                
            # 제목 수집
            tit = a.text
            
            # 개별 기사 클릭
            a.send_keys(Keys.ENTER)
                
            # 본문 수집
            body = WebDriverWait(driver, 10).until(lambda x : x.find_element_by_css_selector("#article_body")).text
                
            # 데이터 프레임에 삽입
            education_df.loc[article_cnt] = [date, tit, body, 2]
            article_cnt+=1
            
            print(date)
            print(tit)
            print("------------------------------------------------")
                
            # 2500개가 돼도 페이지 안의 기사 수 만큼 for문을 돌기 때문에 2500개일 때 break
            if article_cnt==2500:
                break
                
            driver.back()
            driver.implicitly_wait(3)
            

        p+=1
        if article_cnt==2500:
                break

    if article_cnt==2500:
                break
    # 이전 날짜로 가는 버튼 클릭
    action=ActionChains(driver)
    #h2 = WebDriverWait(driver,10).until(lambda x: x.find_element_by_css_selector("body > div.doc > header > div.mh > div > h2"))
    driver.execute_script("window.scrollTo(0, 0)")
    btn = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_prev")))
    #print(btn)
    btn.send_keys(Keys.ENTER)
    date = WebDriverWait(driver,10).until(lambda x : x.find_element_by_id("display_search_date")).text.replace(".","-")
    
    #date_cnt+=1

driver.close()

2020-04-26
원격수업에 불안한 고3 수험생들 “수시‧정시 택일해 집중해야”
------------------------------------------------
2020-04-26
기준금리 반토막인데…학자금 금리 '찔끔 인하'에 생색낸 교육부
------------------------------------------------
2020-04-26
"스타강사 되려 대학 졸업도 포기" '일타' 꿈꾸는 삽자루 제자들
------------------------------------------------
2020-04-26
"1년 견디면 일타강사"···직장 그만 두고 삽자루 밑으로 갔다
------------------------------------------------
2020-04-25
[사진] 차 타고 시험지 수령
------------------------------------------------
2020-04-24
서울신학대학교, 전체 재학생 코로나19 특별장학금 10만원 지급
------------------------------------------------
2020-04-24
‘코로나19’로 인한 온라인 논문이용불편 해결에 학술기업이 나선다
------------------------------------------------
2020-04-24
재능교육, 코로나19 대응 위해 해외 지사 온라인 교육 서비스 오픈
------------------------------------------------
2020-04-24
숙명여대, 전체 구성원 참여하는 총장 직선제 실시
------------------------------------------------
2020-04-24
고3보다 빡빡한 스타강사 양성소…삽자루 "드립도 훈련하라"
------------------------------------------------
2020-04-24
[영상]'삽자루'가 말하는 일타강사의 조건 "드립도 연습하라"
----------------------

In [73]:
education_df

,date,title,body,category
0,2020-04-26,원격수업에 불안한 고3 수험생들 “수시‧정시 택일해 집중해야”,지난 9일 오전 부산에 거주하는 고3 한 학생이 집에서 원격수업을 듣고 있다. 뉴스...,2
1,2020-04-26,기준금리 반토막인데…학자금 금리 '찔끔 인하'에 생색낸 교육부,유은혜 부총리 겸 교육부 장관이 8일 경기도 고양시 EBS를 찾아 신종 코로나바이러...,2
2,2020-04-26,"""스타강사 되려 대학 졸업도 포기"" '일타' 꿈꾸는 삽자루 제자들",사교육 업계에서 막대한 부와 인기를 누리는 일타강사. 업체들은 수백억의 계약금을 감...,2
3,2020-04-26,"""1년 견디면 일타강사""···직장 그만 두고 삽자루 밑으로 갔다",추천영상 더보기\n사교육 업계에서 막대한 부와 인기를 누리는 일타강사. 업체들은 수...,2
4,2020-04-25,[사진] 차 타고 시험지 수령,차 타고 시험지 수령\n코로나19 확산 우려로 연기했던 올해 첫 전국연합학력평가가 ...,2
...,...,...,...,...
97,2020-04-17,"세종대 교수학습개발센터, 비대면 온라인 학습법 특강 진행",세종대학교(총장 배덕효) 교수학습개발센터는 7월 31일까지 온라인 강의 플랫폼인 블...,2
98,2020-04-17,삼육대 기술지주자회사 앤투비 ‘나노버블 기술’ 첫 상용화,"오른쪽부터 삼육대 기술지주자회사 앤투비 박명환(화학생명과학과 교수) 대표, 팬터메딕...",2
99,2020-04-17,"상명대 기장서 교수 연구팀, 해양 독성 플랑크톤의 패류독소 합성능력 차이 규명",상명대학교(총장 백웅기) 융합공과대학 생명공학전공 기장서 교수 연구팀은 국내 패류독...,2
100,2020-04-17,[사진] “얘들아 반갑다” 초등교도 온라인 개학,’얘들아 반갑다“ 초등교도 온라인 개학\n2차 온라인 개학을 한 16일 서울 신동초...,2


In [74]:
education_df.to_csv('D:/Users/HYUN/Desktop/News_Timeline/data/education_df_6.csv',encoding='utf-8-sig')

In [75]:
import pandas as pd

sum=0
for i in range(1, 7):
    df = pd.read_csv('D:/Users/HYUN/Desktop/News_Timeline/data/education_df_{}.csv'.format(i))
    sum += len(df)
    
print(sum)

2508


## 3. 사건사고

In [ ]:
# 데이터 프레임 생성
accident_df = DataFrame(columns=['date', 'title','body', 'category'])

# 드라이버 지정
chrome_path = '/opt/homebrew/bin/chromedriver'
driver = webdriver.Chrome(chrome_path)

# 맨 처음 검색할 날짜 지정
date = "2020-12-31"

# 몇일동안 크롤링할지
#date_cnt = 0

# 수집한 기사 개수 카운트
article_cnt = 0

while (article_cnt < 2500):
    # 먼저 초기화하고 while문 안에서 page수 계산한 다음 그만큼 돈다
    p = 1
    pages = 1
    # 해당 날짜에 나온 기사의 여러 페이지 반복탐색
    while (p<=pages):
        url = "https://news.joins.com/society/accident/list/{}?filter=OnlyJoongang&Date={}".format(p, date)
        driver.get(url)
#         driver.maximize_window()
        response = requests.get(url=url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        pages = len(driver.find_elements_by_class_name("link_page"))
        
        # t_len: 한 페이지 안의 총 기사 개수
        t_len = len(driver.find_elements_by_css_selector("#content > div.list_basic > ul > li"))
        
        # 한 페이지 안의 기사 수 만큼 탐색
        for i in range(1, t_len+1):
            if t_len==1:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li > h2 > a")
            else:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li:nth-child({}) > h2 > a".format(i))
                
            # 제목 수집
            tit = a.text
            
            # 개별 기사 클릭
            a.send_keys(Keys.ENTER)
                
            # 본문 수집
            body = WebDriverWait(driver, 10).until(lambda x : x.find_element_by_css_selector("#article_body")).text
                
            # 데이터 프레임에 삽입
            accident_df.loc[article_cnt] = [date, tit, body, 3]
            article_cnt+=1
            
            print(date)
            print(tit)
            print("------------------------------------------------")
                
            # 2500개가 돼도 페이지 안의 기사 수 만큼 for문을 돌기 때문에 2500개일 때 break
            if article_cnt==2500:
                break
                
            driver.back()
            driver.implicitly_wait(3)
            

        p+=1
        if article_cnt==2500:
                break

    if article_cnt==2500:
                break
    # 이전 날짜로 가는 버튼 클릭
    action=ActionChains(driver)
    #h2 = WebDriverWait(driver,10).until(lambda x: x.find_element_by_css_selector("body > div.doc > header > div.mh > div > h2"))
    driver.execute_script("window.scrollTo(0, 0)")
    btn = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_prev")))
    #print(btn)
    btn.send_keys(Keys.ENTER)
    date = WebDriverWait(driver,10).until(lambda x : x.find_element_by_id("display_search_date")).text.replace(".","-")
    
    #date_cnt+=1

driver.close()

accident_df.to_csv(save_dir+'accident_df')

## 4. 복지

In [ ]:
# 데이터 프레임 생성
welfare_df = DataFrame(columns=['date', 'title','body', 'category'])

# 드라이버 지정
chrome_path = '/opt/homebrew/bin/chromedriver'
driver = webdriver.Chrome(chrome_path)

# 맨 처음 검색할 날짜 지정
date = "2020-12-31"

# 몇일동안 크롤링할지
#date_cnt = 0

# 수집한 기사 개수 카운트
article_cnt = 0

while (article_cnt < 2500):
    # 먼저 초기화하고 while문 안에서 page수 계산한 다음 그만큼 돈다
    p = 1
    pages = 1
    # 해당 날짜에 나온 기사의 여러 페이지 반복탐색
    while (p<=pages):
        url = "https://news.joins.com/society/welfare/list/{}?filter=OnlyJoongang&Date={}".format(p, date)
        driver.get(url)
#         driver.maximize_window()
        response = requests.get(url=url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        pages = len(driver.find_elements_by_class_name("link_page"))
        
        # t_len: 한 페이지 안의 총 기사 개수
        t_len = len(driver.find_elements_by_css_selector("#content > div.list_basic > ul > li"))
        
        # 한 페이지 안의 기사 수 만큼 탐색
        for i in range(1, t_len+1):
            if t_len==1:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li > h2 > a")
            else:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li:nth-child({}) > h2 > a".format(i))
                
            # 제목 수집
            tit = a.text
            
            # 개별 기사 클릭
            a.send_keys(Keys.ENTER)
                
            # 본문 수집
            body = WebDriverWait(driver, 10).until(lambda x : x.find_element_by_css_selector("#article_body")).text
                
            # 데이터 프레임에 삽입
            welfare_df.loc[article_cnt] = [date, tit, body, 4]
            article_cnt+=1
            
            print(date)
            print(tit)
            print("------------------------------------------------")
                
            # 2500개가 돼도 페이지 안의 기사 수 만큼 for문을 돌기 때문에 2500개일 때 break
            if article_cnt==2500:
                break
                
            driver.back()
            driver.implicitly_wait(3)
            

        p+=1
        if article_cnt==2500:
                break

    if article_cnt==2500:
                break
    # 이전 날짜로 가는 버튼 클릭
    action=ActionChains(driver)
    #h2 = WebDriverWait(driver,10).until(lambda x: x.find_element_by_css_selector("body > div.doc > header > div.mh > div > h2"))
    driver.execute_script("window.scrollTo(0, 0)")
    btn = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_prev")))
    #print(btn)
    btn.send_keys(Keys.ENTER)
    date = WebDriverWait(driver,10).until(lambda x : x.find_element_by_id("display_search_date")).text.replace(".","-")
    
    #date_cnt+=1

driver.close()

welfare_df.to_csv(save_dir+'welfare_df')

## 5. 교통

In [ ]:
# 데이터 프레임 생성
traffic_df = DataFrame(columns=['date', 'title','body', 'category'])

# 드라이버 지정
chrome_path = '/opt/homebrew/bin/chromedriver'
driver = webdriver.Chrome(chrome_path)

# 맨 처음 검색할 날짜 지정
date = "2020-12-31"

# 몇일동안 크롤링할지
#date_cnt = 0

# 수집한 기사 개수 카운트
article_cnt = 0

while (article_cnt < 2500):
    # 먼저 초기화하고 while문 안에서 page수 계산한 다음 그만큼 돈다
    p = 1
    pages = 1
    # 해당 날짜에 나온 기사의 여러 페이지 반복탐색
    while (p<=pages):
        url = "https://news.joins.com/society/traffic/list/{}?filter=OnlyJoongang&Date={}".format(p, date)
        driver.get(url)
#         driver.maximize_window()
        response = requests.get(url=url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        pages = len(driver.find_elements_by_class_name("link_page"))
        
        # t_len: 한 페이지 안의 총 기사 개수
        t_len = len(driver.find_elements_by_css_selector("#content > div.list_basic > ul > li"))
        
        # 한 페이지 안의 기사 수 만큼 탐색
        for i in range(1, t_len+1):
            if t_len==1:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li > h2 > a")
            else:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li:nth-child({}) > h2 > a".format(i))
                
            # 제목 수집
            tit = a.text
            
            # 개별 기사 클릭
            a.send_keys(Keys.ENTER)
                
            # 본문 수집
            body = WebDriverWait(driver, 10).until(lambda x : x.find_element_by_css_selector("#article_body")).text
                
            # 데이터 프레임에 삽입
            traffic_df.loc[article_cnt] = [date, tit, body, 5]
            article_cnt+=1
            
            print(date)
            print(tit)
            print("------------------------------------------------")
                
            # 2500개가 돼도 페이지 안의 기사 수 만큼 for문을 돌기 때문에 2500개일 때 break
            if article_cnt==2500:
                break
                
            driver.back()
            driver.implicitly_wait(3)
            

        p+=1
        if article_cnt==2500:
                break

    if article_cnt==2500:
                break
    # 이전 날짜로 가는 버튼 클릭
    action=ActionChains(driver)
    #h2 = WebDriverWait(driver,10).until(lambda x: x.find_element_by_css_selector("body > div.doc > header > div.mh > div > h2"))
    driver.execute_script("window.scrollTo(0, 0)")
    btn = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_prev")))
    #print(btn)
    btn.send_keys(Keys.ENTER)
    date = WebDriverWait(driver,10).until(lambda x : x.find_element_by_id("display_search_date")).text.replace(".","-")
    
    #date_cnt+=1

driver.close()

traffic_df.to_csv(save_dir+'traffic_df')

## 6. 환경

In [ ]:
# 데이터 프레임 생성
environment_df = DataFrame(columns=['date', 'title','body', 'category'])

# 드라이버 지정
chrome_path = '/opt/homebrew/bin/chromedriver'
driver = webdriver.Chrome(chrome_path)

# 맨 처음 검색할 날짜 지정
date = "2020-12-31"

# 몇일동안 크롤링할지
#date_cnt = 0

# 수집한 기사 개수 카운트
article_cnt = 0

while (article_cnt < 2500):
    # 먼저 초기화하고 while문 안에서 page수 계산한 다음 그만큼 돈다
    p = 1
    pages = 1
    # 해당 날짜에 나온 기사의 여러 페이지 반복탐색
    while (p<=pages):
        url = "https://news.joins.com/society/environment/list/{}?filter=OnlyJoongang&Date={}".format(p, date)
        driver.get(url)
#         driver.maximize_window()
        response = requests.get(url=url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        pages = len(driver.find_elements_by_class_name("link_page"))
        
        # t_len: 한 페이지 안의 총 기사 개수
        t_len = len(driver.find_elements_by_css_selector("#content > div.list_basic > ul > li"))
        
        # 한 페이지 안의 기사 수 만큼 탐색
        for i in range(1, t_len+1):
            if t_len==1:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li > h2 > a")
            else:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li:nth-child({}) > h2 > a".format(i))
                
            # 제목 수집
            tit = a.text
            
            # 개별 기사 클릭
            a.send_keys(Keys.ENTER)
                
            # 본문 수집
            body = WebDriverWait(driver, 10).until(lambda x : x.find_element_by_css_selector("#article_body")).text
                
            # 데이터 프레임에 삽입
            environment_df.loc[article_cnt] = [date, tit, body, 6]
            article_cnt+=1
            
            print(date)
            print(tit)
            print("------------------------------------------------")
                
            # 2500개가 돼도 페이지 안의 기사 수 만큼 for문을 돌기 때문에 2500개일 때 break
            if article_cnt==2500:
                break
                
            driver.back()
            driver.implicitly_wait(3)
            

        p+=1
        if article_cnt==2500:
                break

    if article_cnt==2500:
                break
    # 이전 날짜로 가는 버튼 클릭
    action=ActionChains(driver)
    #h2 = WebDriverWait(driver,10).until(lambda x: x.find_element_by_css_selector("body > div.doc > header > div.mh > div > h2"))
    driver.execute_script("window.scrollTo(0, 0)")
    btn = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_prev")))
    #print(btn)
    btn.send_keys(Keys.ENTER)
    date = WebDriverWait(driver,10).until(lambda x : x.find_element_by_id("display_search_date")).text.replace(".","-")
    
    #date_cnt+=1

driver.close()

environment_df.to_csv(save_dir+'environment_df')

## 7. 지역

In [ ]:
# 데이터 프레임 생성
region_df = DataFrame(columns=['date', 'title','body', 'category'])

# 드라이버 지정
chrome_path = '/opt/homebrew/bin/chromedriver'
driver = webdriver.Chrome(chrome_path)

# 맨 처음 검색할 날짜 지정
date = "2020-12-31"

# 몇일동안 크롤링할지
#date_cnt = 0

# 수집한 기사 개수 카운트
article_cnt = 0

while (article_cnt < 2500):
    # 먼저 초기화하고 while문 안에서 page수 계산한 다음 그만큼 돈다
    p = 1
    pages = 1
    # 해당 날짜에 나온 기사의 여러 페이지 반복탐색
    while (p<=pages):
        url = "https://news.joins.com/society/region/list/{}?filter=OnlyJoongang&Date={}".format(p, date)
        driver.get(url)
#         driver.maximize_window()
        response = requests.get(url=url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        pages = len(driver.find_elements_by_class_name("link_page"))
        
        # t_len: 한 페이지 안의 총 기사 개수
        t_len = len(driver.find_elements_by_css_selector("#content > div.list_basic > ul > li"))
        
        # 한 페이지 안의 기사 수 만큼 탐색
        for i in range(1, t_len+1):
            if t_len==1:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li > h2 > a")
            else:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li:nth-child({}) > h2 > a".format(i))
                
            # 제목 수집
            tit = a.text
            
            # 개별 기사 클릭
            a.send_keys(Keys.ENTER)
                
            # 본문 수집
            body = WebDriverWait(driver, 10).until(lambda x : x.find_element_by_css_selector("#article_body")).text
                
            # 데이터 프레임에 삽입
            region_df.loc[article_cnt] = [date, tit, body, 7]
            article_cnt+=1
            
            print(date)
            print(tit)
            print("------------------------------------------------")
                
            # 2500개가 돼도 페이지 안의 기사 수 만큼 for문을 돌기 때문에 2500개일 때 break
            if article_cnt==2500:
                break
                
            driver.back()
            driver.implicitly_wait(3)
            

        p+=1
        if article_cnt==2500:
                break

    if article_cnt==2500:
                break
    # 이전 날짜로 가는 버튼 클릭
    action=ActionChains(driver)
    #h2 = WebDriverWait(driver,10).until(lambda x: x.find_element_by_css_selector("body > div.doc > header > div.mh > div > h2"))
    driver.execute_script("window.scrollTo(0, 0)")
    btn = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_prev")))
    #print(btn)
    btn.send_keys(Keys.ENTER)
    date = WebDriverWait(driver,10).until(lambda x : x.find_element_by_id("display_search_date")).text.replace(".","-")
    
    #date_cnt+=1

driver.close()

region_df.to_csv(save_dir+'region_df')

## 8. 건강

In [ ]:
# 데이터 프레임 생성
health_df = DataFrame(columns=['date', 'title','body', 'category'])

# 드라이버 지정
chrome_path = '/opt/homebrew/bin/chromedriver'
driver = webdriver.Chrome(chrome_path)

# 맨 처음 검색할 날짜 지정
date = "2020-12-31"

# 몇일동안 크롤링할지
#date_cnt = 0

# 수집한 기사 개수 카운트
article_cnt = 0

while (article_cnt < 2500):
    # 먼저 초기화하고 while문 안에서 page수 계산한 다음 그만큼 돈다
    p = 1
    pages = 1
    # 해당 날짜에 나온 기사의 여러 페이지 반복탐색
    while (p<=pages):
        url = "https://news.joins.com/society/region/list/{}?filter=OnlyJoongang&Date={}".format(p, date)
        driver.get(url)
#         driver.maximize_window()
        response = requests.get(url=url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        pages = len(driver.find_elements_by_class_name("link_page"))
        
        # t_len: 한 페이지 안의 총 기사 개수
        t_len = len(driver.find_elements_by_css_selector("#content > div.list_basic > ul > li"))
        
        # 한 페이지 안의 기사 수 만큼 탐색
        for i in range(1, t_len+1):
            if t_len==1:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li > h2 > a")
            else:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li:nth-child({}) > h2 > a".format(i))
                
            # 제목 수집
            tit = a.text
            
            # 개별 기사 클릭
            a.send_keys(Keys.ENTER)
                
            # 본문 수집
            body = WebDriverWait(driver, 10).until(lambda x : x.find_element_by_css_selector("#article_body")).text
                
            # 데이터 프레임에 삽입
            health_df.loc[article_cnt] = [date, tit, body, 8]
            article_cnt+=1
            
            print(date)
            print(tit)
            print("------------------------------------------------")
                
            # 2500개가 돼도 페이지 안의 기사 수 만큼 for문을 돌기 때문에 2500개일 때 break
            if article_cnt==2500:
                break
                
            driver.back()
            driver.implicitly_wait(3)
            

        p+=1
        if article_cnt==2500:
                break

    if article_cnt==2500:
                break
    # 이전 날짜로 가는 버튼 클릭
    action=ActionChains(driver)
    #h2 = WebDriverWait(driver,10).until(lambda x: x.find_element_by_css_selector("body > div.doc > header > div.mh > div > h2"))
    driver.execute_script("window.scrollTo(0, 0)")
    btn = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_prev")))
    #print(btn)
    btn.send_keys(Keys.ENTER)
    date = WebDriverWait(driver,10).until(lambda x : x.find_element_by_id("display_search_date")).text.replace(".","-")
    
    #date_cnt+=1

driver.close()

health_df.to_csv(save_dir+'health_df')